## 本教程是复现[《Different fibroblast subtypes propel spatially defined ileal inflammation through TNFR1 signalling in murine ileitis》](https://www.nature.com/articles/s41467-025-57570-7)这篇文章中的数据分析过程。
- 脚本下载：[github](https://github.com/ChrisTzaferis/Iliopoulou_et_Al_Wt_Dare12wk_sc_analysis.git)

# Step2_InitialSeuratObjects

```r
library(Seurat)
library(patchwork)
library(dplyr)

#object1 initialization
obj1_counts <- Read10X("../GEO_files/PROCESSED/WT12/")
obj1 <- CreateSeuratObject(counts = obj1_counts, project = "WT12wk",
                           min.cells = 0, min.features = 0)
# 返回一个稀疏计数矩阵，其中行代表基因，列代表细胞条形码（barcode），
# 值代表该基因在该细胞中检测到的分子数（UMI计数）
obj1[["percent.mt"]] <- PercentageFeatureSet(obj1, pattern = "^mt-")
# 线粒体基因编码的RNA占比。
# 线粒体基因高表达通常指示细胞凋亡或损伤。
# 小鼠线粒体基因以"mt-"开头，人类以"MT-"开头
obj1$sample <- "wt-12w"
obj1$genotype <- "wt"
head(obj1@meta.data)

#doublet removal
#load scrublet results
# 双细胞问题：​​ 微流体实验中两个细胞可能被包裹在同一个液滴中，形成"双细胞"，会误导下游分析
obj1_doublets <- read.csv("../Step1_DoubletRemovalWithScrublet/WT12_scrublet_results.csv")
colnames(obj1_doublets)[1] <- "Cell_id"
obj1_doublets <- obj1_doublets[, c("Cell_id", "predicted_doublets")]

metadata_temp1 <- as.data.frame(obj1@meta.data)
metadata_temp1$Cell_id <- rownames(metadata_temp1)
metadata_temp1 <- left_join(metadata_temp1, obj1_doublets)
rownames(metadata_temp1) <- metadata_temp1$Cell_id

obj1@meta.data <- metadata_temp1
Idents(obj1) <- obj1$predicted_doublets

#View QC plots before doublet removal
VlnPlot(obj1, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"),
        ncol = 3, group.by = "orig.ident", pt.size = 0)

# 三个关键质控指标：​​
# ​nFeature_RNA​：每个细胞检测到的基因数
# ​nCount_RNA​：每个细胞的总UMI计数（转录本数）
# ​percent.mt​：线粒体基因占比
# 质控阈值判断标准
# 
# ​nFeature_RNA过低​：可能为空液滴或死细胞
# ​nFeature_RNA过高​：可能为多个细胞形成的双细胞
# ​percent.mt过高​：通常>5-10%表示细胞状态不佳

#Remove doublets
obj1_no_doublets <- subset(obj1, idents = "False")

obj1 #43119 cells
obj1_no_doublets #35887 cells

#View QC plots after doublet removal
VlnPlot(obj1_no_doublets, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, group.by = "orig.ident", pt.size = 0)
saveRDS(obj1_no_doublets, "../output/wt12_no_doubletsDefaultThreshold_not_filtered.RDS")

#object2 initialization
obj2_counts <- Read10X("../GEO_files/PROCESSED/DARE12/")
obj2 <- CreateSeuratObject(counts = obj2_counts, project = "DARE12wk", 
                           min.cells = 0, min.features = 0)
obj2[["percent.mt"]] <- PercentageFeatureSet(obj2, pattern = "^mt-")
obj2$sample <- "dare-12w"
obj2$genotype <- "dare"

#doublet removal
#load scrublet results
obj2_doublets <- read.csv("../Step1_DoubletRemovalWithScrublet/DARE12_scrublet_results.csv")
colnames(obj2_doublets)[1] <- "Cell_id"
obj2_doublets <- obj2_doublets[, c("Cell_id", "predicted_doublets")]

metadata_temp2 <- as.data.frame(obj2@meta.data)
metadata_temp2$Cell_id <- rownames(metadata_temp2)
metadata_temp2 <- left_join(metadata_temp2, obj2_doublets)
rownames(metadata_temp2) <- metadata_temp2$Cell_id

obj2@meta.data <- metadata_temp2
Idents(obj2) <- obj2$predicted_doublets

#View QC plots before doublet removal
VlnPlot(obj2, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, group.by = "orig.ident", pt.size = 0)

#Remove doublets
obj2_no_doublets <- subset(obj2, idents = "False")

obj2 #25771 cells
obj2_no_doublets #22708 cells

#View QC plots after doublet removal
VlnPlot(obj2_no_doublets, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, group.by = "orig.ident", pt.size = 0)
saveRDS(obj2_no_doublets, "../output/dare12_no_doubletsDefaultThreshold_not_filtered.RDS")
```

## 解读Seurat对象 
    要把Seurat对象，当做一个数据库，包含多种数据
    其中，最重要的就是包含了多个表达矩阵和细胞注释信息（类似于临床信息）
    当需要用到表达矩阵或细胞注释信息时，会使用默认信息
    不想要使用默认信息，就必须指定或修改默认信息
    用@,$符号依次取下层，也可以[[]]
    meta.data：    储存着细胞注释信息（类似于临床信息）
    active.ident： 储存着默认的细胞注释信息（类似于临床信息）
    active.assay： 储存着默认的矩阵名
    assays：       储存着表达矩阵
    counts：       存储原始数据，是稀疏矩阵
    data：         存储Normalize() 规范化的data
    scale.data：   存储 ScaleData()缩放后的data
    SCT：          储存SCT标准化之后的data

## 其他格式的读取

### H5格式

```r
setwd("GSE199866")
fs=list.files(pattern = '.h5')
fs
sceList = lapply(fs, function(x){
    a=Read10X_h5(x)
    p=strsplit(x,'',simplify = T)[,1]
    sce <- CreateSeuratObject(a,project = p ,min.features = 200,min.cells = 3)
})
folders = substr(fs,1,10)
folders
sce.big <- merge(sceList[[1]],  #使用merge函数进行合并
    y = sceList[-1],
    add.cell.ids = folders)
pbmc = JoinLayers(sce.big)#将Layers融合
```

### R数据文件(RDS/RDATA文件)

```r
setwd("F:/4scRNA/1code/3/3")
load(file ="1pbmc.RData")#读取RDATA文件
pbmc2 = readRDS("1pbmc.rds")#读取RDS文件

# Step3_SeuratWorkflowPerSample

```r
rm(list=ls())
library(Seurat)
library(ggplot2)
# remotes::install_github("samuel-marsh/scCustomize@v1.1.3")
library(scCustomize)

#load object
obj <- readRDS("../Step2_InitialSeuratObjects/wt12_no_doubletsDefaultThreshold_not_filtered.RDS")
obj

#QC filtering
VlnPlot(obj, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"))
# p <- VlnPlot(obj, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"))
# # 保存为PNG（位图，适合网页展示）
# ggsave(plot = p, filename = "QC_VlnPlot.png", width = 10, height = 6, dpi = 300)

obj_filtered <- subset(obj, subset = nFeature_RNA > 800 & 
                         nFeature_RNA < 7500 & percent.mt < 5)
# nFeature_RNA > 800​：确保细胞有足够的基因表达信息
# ​nFeature_RNA < 7500​：排除可能的多细胞或双细胞
# ​percent.mt < 5​：排除线粒体基因污染严重的低质量细胞
VlnPlot(obj_filtered, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"))
obj_filtered

#wt: 
# before filtering: 35428 cells
# nFeature_RNA > 800 & nFeature_RNA < 7500 & percent.mt < 5: 24249 cells

#normalization
obj_filtered <- NormalizeData(obj_filtered)
# 作用​：消除细胞间测序深度的技术差异。
# ​原理​：默认使用"LogNormalize"方法，将每个细胞的基因表达量除以该细胞的总UMI数，乘以缩放因子10,000，然后进行对数转换
# 。这使得不同细胞间的表达量具有可比性。

#HVGs
obj_filtered <- FindVariableFeatures(obj_filtered, selection.method = "mvp")
# 作用​：识别在细胞间表达变异最大的基因，用于下游分析。
# ​原理​：高变基因通常包含区分不同细胞类型的生物信号，
# 聚焦这些基因可以提高分析效率和准确性。
# "mvp"方法基于平均表达量和离散度来选择基因

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(obj_filtered), 10)

# plot variable features with and without labels
plot1 <- VariableFeaturePlot(obj_filtered)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot1 + plot2

#scaling
#all.genes <- rownames(obj_filtered)
obj_filtered <- ScaleData(obj_filtered, vars.to.regress = c("nCount_RNA", "percent.mt"))
# 作用​：标准化基因表达量，消除技术变异的影响。
# ​原理​：
# 将每个基因的表达量转换为z-score（均值为0，方差为1）
# vars.to.regress参数回归去除nCount_RNA和percent.mt的影响，
# 减少测序深度和线粒体污染带来的技术偏差
# 为PCA分析做准备，确保高表达基因不会主导分析结果

#PCA
obj_filtered <- RunPCA(obj_filtered, features = VariableFeatures(object = obj_filtered))
ElbowPlot(obj_filtered, ndims = 50)

#Clustering
obj_filtered <- FindNeighbors(obj_filtered, dims = 1:20)
obj_filtered <- FindClusters(obj_filtered, resolution = 0.01)
# FindNeighbors：基于PCA空间构建K近邻图，连接相似的细胞
# FindClusters：应用Louvain算法识别细胞群落，resolution参数控制聚类粒度（值越小聚类越少）

#UMAP
obj_filtered <- RunUMAP(obj_filtered, dims = 1:20)
DimPlot(obj_filtered, reduction = "umap", label = T)
# 作用​：非线性降维可视化，保留细胞间的全局和局部结构关系。
# ​原理​：UMAP比t-SNE能更好地保留数据的全局结构，用于直观展示细胞亚群的分布

#annotation of WT
new.cluster.ids <- c("Lymphoid", "Stroma", "Lymphoid", "Myeloid", "Stroma", "Stroma")
names(new.cluster.ids) <- levels(obj_filtered)
obj_filtered <- RenameIdents(obj_filtered, new.cluster.ids)
# 作用​：基于已知的标记基因将聚类结果转换为有生物学意义的细胞类型。
# ​注释依据​：
# ​Lymphoid​：免疫细胞谱系（如T细胞、B细胞）
# ​Myeloid​：髓系免疫细胞（如巨噬细胞、树突状细胞）
# ​Stroma​：基质细胞（如成纤维细胞
#save
saveRDS(obj_filtered, "WT12_weeks_before_integration_nF_800.RDS") 

#used in Fig1b
FeaturePlot(obj_filtered, features = "Itgax", order = T, label = T, cols = c("grey70", "red"))
FeaturePlot(obj_filtered, features = "Itgam", order = T, label = T, cols = c("grey70", "red"))
FeaturePlot(obj_filtered, features = "Ptprc", order = T, label = T, cols = c("grey70", "red"))
FeaturePlot(obj_filtered, features = "Col1a1", order = T, label = T, cols = c("grey70", "red"))
# 验证
# Itgax​（CD11c）：树突状细胞标记
# ​Itgam​（CD11b）：髓系细胞标记
# ​Ptprc​（CD45）：白细胞共同标记
# ​Col1a1​：胶原蛋白，基质细胞标记

DimPlot_scCustom(obj_filtered, colors_use = c("purple", "coral", "grey"), pt.size = 0.8, label.size = 7) + 
  theme(axis.line = element_blank(), 
        axis.text = element_blank(), 
        axis.ticks = element_blank(),
        axis.title = element_blank()
        )
#  分析流程总结
# 这个流程体现了单细胞分析的标准范式：
# 质控过滤→标准化→特征选择→降维聚类→细胞注释
# 每个步骤都为后续分析奠定基础，确保结果的生物学可靠性
# 这种分析能够揭示细胞异质性，识别稀有细胞类型，
# 为理解组织微环境的细胞组成和功能状态提供重要见解。

# Q:高变基因识别之前不对数据进行缩放和批次效应矫正
# 数据标准化在高变基因识别之前；
#     消除细胞间测序深度的技术差异，使表达量可比。为准确评估基因变异提供公平的“起跑线”。
# 而数据缩放和批次效应矫正在后：避免对高变基因选择引入偏差。
#     将基因表达值转换为Z-score，为线性降维（如PCA）做准备。



# Step4_Analysis_of_Stroma_Lymphoid_Myeloid

```r
rm(list=ls())
library(Seurat)

wt12 <- readRDS("../Step3_SeuratWorkflowPerSample/WT12_weeks_before_integration_nF_800.RDS")
dare12 <- readRDS("../Step3_SeuratWorkflowPerSample/DARE12_weeks_before_integration_nF_800.RDS")

analyzeAndSaveObjects <- function(seu_obj, supercluster, nPCs, resolution, sampleName)
{
  obj_filtered <- subset(seu_obj, idents = supercluster)
  # 从完整的Seurat对象中，根据细胞身份（idents）提取出指定大类（如 "Myeloid"）的所有细胞，
  # 创建一个新的、只包含该大类细胞的Seurat对象
  
  #normalization
  obj_filtered <- NormalizeData(obj_filtered)
  
  #HVGs
  obj_filtered <- FindVariableFeatures(obj_filtered, selection.method = "mvp")
  
  # Identify the 10 most highly variable genes
  top10 <- head(VariableFeatures(obj_filtered), 10)
  
  # plot variable features with and without labels
  plot1 <- VariableFeaturePlot(obj_filtered)
  plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
  p1 <- plot1 + plot2
  ggsave(plot = p1, filename = paste0("s4.", sampleName, "_", supercluster, ".VariableFeaturePlot.png"),
                                      width = 12, height = 6, dpi = 300)
  
  #scaling
  #all.genes <- rownames(obj_filtered)
  obj_filtered <- ScaleData(obj_filtered, vars.to.regress = c("nCount_RNA", "percent.mt"))
  
  #PCA
  obj_filtered <- RunPCA(obj_filtered, features = VariableFeatures(object = obj_filtered))
  p2 <- ElbowPlot(obj_filtered, ndims = 50)
  ggsave(plot = p2, filename = paste0("s4.", sampleName, "_", supercluster, ".ElbowPlot.png"),
         width = 10, height = 6, dpi = 300)
  
  #Clustering
  obj_filtered <- FindNeighbors(obj_filtered, dims = 1:nPCs)
  obj_filtered <- FindClusters(obj_filtered, resolution = resolution)
  
  #UMAP
  obj_filtered <- RunUMAP(obj_filtered, dims = 1:nPCs)
  obj_filtered <- RunTSNE(obj_filtered, dims = 1:nPCs)
  p3 <- DimPlot(obj_filtered, reduction = "umap")
  ggsave(plot = p3, filename = paste0("s4.", sampleName, "_", supercluster, ".DimPlot.png"),
         width = 6, height = 6, dpi = 300)
  
  #export RDS
  saveRDS(object = obj_filtered, file = paste0(sampleName, "_", supercluster, "_PCs_", nPCs, "_", resolution, ".RDS")) 
}

analyzeAndSaveObjects(seu_obj = wt12, supercluster = "Myeloid", 
                      nPCs = 14, resolution = 0.5, sampleName = "wt12")
analyzeAndSaveObjects(seu_obj = dare12, supercluster = "Myeloid", 
                      nPCs = 21, resolution = 0.5, sampleName = "dare12")

analyzeAndSaveObjects(seu_obj = wt12, supercluster = "Lymphoid", 
                      nPCs = 15, resolution = 0.5, sampleName = "wt12")
analyzeAndSaveObjects(seu_obj = dare12, supercluster = "Lymphoid", 
                      nPCs = 23, resolution = 0.5, sampleName = "dare12")

analyzeAndSaveObjects(seu_obj = wt12, supercluster = "Stroma", 
                      nPCs = 20, resolution = 0.5, sampleName = "wt12")
analyzeAndSaveObjects(seu_obj = dare12, supercluster = "Stroma", 
                      nPCs = 22, resolution = 0.5, sampleName = "dare12")


# Step5_DoubletFinder

```r
setwd("D:/01_study/03_scRNA-seq/Iliopoulou_et_Al_Wt_Dare12wk_sc_analysis/analysis/Step5_DoubletFinder/")
rm(list=ls())
library(Seurat)
library(DoubletFinder)
library(dplyr)
library(ggplot2)

## Pre-process Seurat object (standard) --------------------------------------------------------------------------------------
seu <- readRDS("../Step4_Analysis_of_Stroma_Lymphoid_Myeloid/wt12_Stroma_PCs_20_0.5.RDS")

## pK Identification (no ground-truth) ---------------------------------------------------------------------------------------
sweep.res.list_seu <- paramSweep(seu, PCs = 1:20, sct = FALSE)
sweep.stats_seu <- summarizeSweep(sweep.res.list_seu, GT = FALSE)
bcmvn_seu <- find.pK(sweep.stats_seu)

bcmvn_seu %>%
  ggplot( aes(x=pK, y=BCmetric, group=1)) +
  geom_line( color="grey") +
  geom_point(shape=21, color="black", fill="#69b3a2", size=2) +
  theme_bw() +
  ggtitle("pK selection")

## Homotypic Doublet Proportion Estimate -------------------------------------------------------------------------------------
homotypic.prop <- modelHomotypic(seu@meta.data$RNA_snn_res.0.5)
nExp_poi <- round(0.046*nrow(seu@meta.data))
nExp_poi.adj <- round(nExp_poi*(1-homotypic.prop))

## Run DoubletFinder with varying classification stringencies ----------------------------------------------------------------
seu <- doubletFinder(seu, PCs = 1:20, pN = 0.25, pK = 0.3, nExp = nExp_poi, reuse.pANN = FALSE, sct = FALSE)
seu <- doubletFinder(seu, PCs = 1:20, pN = 0.25, pK = 0.3, nExp = nExp_poi.adj, reuse.pANN = "pANN_0.25_0.3_453", sct = FALSE)

seu$DF.classifications_0.25_0.3_453 <- gsub(x = seu$DF.classifications_0.25_0.3_453, pattern = "Doublet", replacement = TRUE)
seu$DF.classifications_0.25_0.3_453 <- gsub(x = seu$DF.classifications_0.25_0.3_453, pattern = "Singlet", replacement = FALSE)
seu$DF.classifications_0.25_0.3_397 <- gsub(x = seu$DF.classifications_0.25_0.3_397, pattern = "Doublet", replacement = TRUE)
seu$DF.classifications_0.25_0.3_397 <- gsub(x = seu$DF.classifications_0.25_0.3_397, pattern = "Singlet", replacement = FALSE)

seu$DoubletFinderClass <- as.logical(seu$DF.classifications_0.25_0.3_453) | as.logical(seu$DF.classifications_0.25_0.3_397)

DimPlot(seu, group.by = "DoubletFinderClass", split.by = "DoubletFinderClass")
saveRDS(seu, "wt12_Stroma_PCs_20_0.05_DoubletFinder.RDS")

################################################################################
################################################################################
rm(list=ls(all=TRUE))#remove previous vars, objects

## Pre-process Seurat object (standard) --------------------------------------------------------------------------------------
seu <- readRDS("../Step4_Analysis_of_Stroma_Lymphoid_Myeloid/dare12_Stroma_PCs_22_0.5.RDS")

## pK Identification (no ground-truth) ---------------------------------------------------------------------------------------
sweep.res.list_seu <- paramSweep(seu, PCs = 1:22, sct = FALSE)
sweep.stats_seu <- summarizeSweep(sweep.res.list_seu, GT = FALSE)
bcmvn_seu <- find.pK(sweep.stats_seu)

bcmvn_seu %>%
  ggplot( aes(x=pK, y=BCmetric, group=1)) +
  geom_line( color="grey") +
  geom_point(shape=21, color="black", fill="#69b3a2", size=2) +
  theme_bw() +
  ggtitle("pK selection")

## Homotypic Doublet Proportion Estimate -------------------------------------------------------------------------------------
homotypic.prop <- modelHomotypic(seu@meta.data$RNA_snn_res.0.5)           
nExp_poi <- round(0.046*nrow(seu@meta.data))
nExp_poi.adj <- round(nExp_poi*(1-homotypic.prop))

## Run DoubletFinder with varying classification stringencies ----------------------------------------------------------------
seu <- doubletFinder(seu, PCs = 1:22, pN = 0.25, pK = 0.25, nExp = nExp_poi, reuse.pANN = FALSE, sct = FALSE)
seu <- doubletFinder(seu, PCs = 1:22, pN = 0.25, pK = 0.25, nExp = nExp_poi.adj, reuse.pANN = "pANN_0.25_0.25_220", sct = FALSE)

seu$DF.classifications_0.25_0.25_220 <- gsub(x = seu$DF.classifications_0.25_0.25_220, pattern = "Doublet", replacement = TRUE)
seu$DF.classifications_0.25_0.25_220 <- gsub(x = seu$DF.classifications_0.25_0.25_220, pattern = "Singlet", replacement = FALSE)
seu$DF.classifications_0.25_0.25_191 <- gsub(x = seu$DF.classifications_0.25_0.25_191, pattern = "Doublet", replacement = TRUE)
seu$DF.classifications_0.25_0.25_191 <- gsub(x = seu$DF.classifications_0.25_0.25_191, pattern = "Singlet", replacement = FALSE)

seu$DoubletFinderClass <- as.logical(seu$DF.classifications_0.25_0.25_220) | as.logical(seu$DF.classifications_0.25_0.25_191)

DimPlot(seu, group.by = "DoubletFinderClass", split.by = "DoubletFinderClass")
saveRDS(seu, "dare12_Stroma_PCs_22_0.05_DoubletFinder.RDS")


# Step6_Integration

```R
# 这份代码展示了单细胞RNA测序数据分析中一个非常关键的步骤——多样本数据整合（Integration）​。
# 简单来说，它的核心目标是消除因实验批次、不同样本来源（如野生型WT vs 基因编辑型DARE）
# 带来的技术差异，使得不同样本间相同类型的细胞能够聚集在一起，
# 从而让我们能够真实地比较生物学条件（如基因型）对细胞产生的影响

setwd("D:/01_study/03_scRNA-seq/Iliopoulou_et_Al_Wt_Dare12wk_sc_analysis/analysis/Step6_Integration/")
rm(list=ls())
library(Seurat)

#Load objects
# 代码首先加载了6个已经过前期处理的Seurat对象，
# 它们分别来自野生型（WT）​​ 和 ​基因编辑型（DARE）​​ 小鼠样本，
# 并已根据主要细胞谱系（淋巴系Lym、髓系Mye、基质Str）进行了分群。
# 这种分细胞大类进行独立整合的策略非常关键，它能有效避免整合算法将不同谱系的细胞错误地“拉近”，
# 从而更好地保留细胞亚群间的生物学差异
dareLym <- readRDS("../Step5_DoubletFinder/dare12_Lymphoid_PCs_23_0.05_DoubletFinder.RDS")
dareMye <- readRDS("../Step5_DoubletFinder/dare12_Myeloid_PCs_21_0.5_DoubletFinder.RDS")
dareStr <- readRDS("../Step5_DoubletFinder/dare12_Stroma_PCs_22_0.05_DoubletFinder.RDS")

wtLym <- readRDS("../Step5_DoubletFinder/wt12_Lymphoid_PCs_15_0.05_DoubletFinder.RDS")
wtMye <- readRDS("../Step5_DoubletFinder/wt12_Myeloid_PCs_14_0.5_DoubletFinder.RDS")
wtStr <- readRDS("../Step5_DoubletFinder/wt12_Stroma_PCs_20_0.05_DoubletFinder.RDS")

integrateSeuratObjects <- function(obj1, obj2, nPCs, resolution, exportName)
{
  #change Idents and keep only unique cells
  Idents(obj1) <- obj1$DoubletFinderClass
  Idents(obj2) <- obj2$DoubletFinderClass
  obj1 <- subset(obj1, idents = "FALSE")
  obj2 <- subset(obj2, idents = "FALSE")
  
  #pre-processing 
  obj.list <- list(obj1, obj2)
  # normalize and identify variable features for each dataset independently
  # 1. 将两个对象放入列表，并分别对每个进行标准化和寻找高变基因
  obj.list <- lapply(X = obj.list, FUN = function(x) {
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2000)
  })
  
  # select features that are repeatedly variable across datasets for integration
  features <- SelectIntegrationFeatures(object.list = obj.list)
  
  #Perform integration
  obj.anchors <- FindIntegrationAnchors(object.list = obj.list, anchor.features = features)
  # FindIntegrationAnchors函数会寻找两个数据集中那些处于相同生物学状态的细胞对（即“锚点”）。
  # 这些锚点就像是不同地图上的共同地标，后续的整合算法会以它们为参考，
  # 将不同数据集（样本）的细胞正确“对齐”到同一个坐标系中，从而校正批次效应。
  
  # this command creates an 'integrated' data assay
  obj.combined <- IntegrateData(anchorset = obj.anchors)
  
  #Perform integrated analysis
  # specify that we will perform downstream analysis on the corrected data note that the
  # original unmodified data still resides in the 'RNA' assay
  DefaultAssay(obj.combined) <- "integrated"
  # 整合后的数据存储在名为 "integrated"的 assay 中，而原始数据仍保留在 "RNA"assay 里。
  # 后续的所有降维和聚类操作都基于校正后的数据，
  # 这能确保聚类结果由真实的生物学状态驱动，而非技术偏差
  
  # Run the standard workflow for visualization and clustering
  obj.combined <- ScaleData(obj.combined, verbose = T)
  obj.combined <- RunPCA(obj.combined, npcs = 30, verbose = T)
  ElbowPlot(obj.combined, ndims = 30)
  obj.combined <- RunUMAP(obj.combined, reduction = "pca", dims = 1:nPCs)
  obj.combined <- FindNeighbors(obj.combined, reduction = "pca", dims = 1:nPCs)
  obj.combined <- FindClusters(obj.combined, resolution = 0.5)
  
  #save RDS
  saveRDS(object = obj.combined, file = exportName)
}

# 代码最后分别对基质、淋巴和髓系细胞进行了三次独立的整合操作。
# 值得注意的是，每次整合都使用了不同的主成分数（nPCs）​​（23, 15, 20），
# 这可能是研究者根据每个细胞大类数据的复杂程度（通过ElbowPlot判断）进行的个性化设置，
# 目的是捕捉最主要的生物学变异

integrateSeuratObjects(obj1 = wtStr, obj2 = dareStr, nPCs = 23, resolution = 0.5, exportName = "WT12wk_DARE12Wk_Integration_Stroma_VST_without_DoubletsFromDoubletFinder.RDS")
integrateSeuratObjects(obj1 = wtLym, obj2 = dareLym, nPCs = 15, resolution = 0.5, exportName = "WT12wk_DARE12Wk_Integration_Lymphoid_VST_without_DoubletsFromDoubletFinder.RDS")
integrateSeuratObjects(obj1 = wtMye, obj2 = dareMye, nPCs = 20, resolution = 0.5, exportName = "WT12wk_DARE12Wk_Integration_Myeloid_VST_without_DoubletsFromDoubletFinder.RDS")

# 这段代码实现了一个稳健、高效的单细胞数据整合流程。其最大价值在于：
# ​消除批次效应​：通过CCA等方法识别生物学上的“锚点”，
#     有效校正了不同样本（WT vs DARE）之间的技术差异。
# ​实现可比性​：使得我们能够在一个统一的空间中，
#     公平地比较不同基因型背景下，同一细胞类型的基因表达差异，为发现真正的生物学现象奠定了基础
# 希望这份详细的解读能帮助你深入理解单细胞数据整合的原理和操作！
# 如果你对某个具体步骤还有疑问，我们可以继续探讨

# Step7_Annotation_of_clusters

```r

setwd("D:/01_study/03_scRNA-seq/Iliopoulou_et_Al_Wt_Dare12wk_sc_analysis/analysis/Step7_Annotation_of_clusters/")
rm(list=ls())

library(Seurat)
library(scCustomize)
library(ggplot2)

#-----------------------Annotation of STROMA cells------------------------------
stroma <- readRDS("../Step6_Integration/WT12wk_DARE12Wk_Integration_Stroma_VST_without_DoubletsFromDoubletFinder.RDS")
DefaultAssay(stroma) <- "integrated"

#Exclusion of clusters 10, 14, 19
stroma <- subset(x = stroma, idents = c(10, 14, 19), invert = T)
stroma <- FindNeighbors(stroma, reduction = "pca", dims = 1:23)
stroma <- FindClusters(stroma, resolution = 0.4)

DimPlot(stroma, reduction = "umap")
FeaturePlot(stroma, features = "Pdgfra_low", order = T, 
            label = T, cols = c("grey70", "red"))

#Annotation of clusters after manual inspection of marker genes from the literature
new.cluster.ids <- c("Pdgfra_low","Pdgfra_low", "LECS", "Trophocytes", "Telocytes", 
                     "SMCs","FRC2", "FRC1", "Capillary_ECs", "Trophocytes", "Pericytes", 
                     "FDCs_MRCs","Cajal", "Vein_ECs", "Mesothelial", "Glial")
names(new.cluster.ids) <- levels(stroma)
stroma <- RenameIdents(object = stroma, new.cluster.ids)
stroma$annotation <- Idents(stroma)
saveRDS(stroma, "WT12wk_DARE12Wk_Stroma_annotated.RDS")
#-------------------------------------------------------------------------------


#-----------------------Annotation of Lymphoid cells----------------------------
lymphoid <- readRDS("../Step6_Integration/WT12wk_DARE12Wk_Integration_Lymphoid_VST_without_DoubletsFromDoubletFinder.RDS")
DefaultAssay(lymphoid) <- "integrated"
lymphoid <- FindClusters(lymphoid, resolution = 0.7)
DimPlot_scCustom(lymphoid)

#Exclusion of cluster 16
lymphoid <- subset(lymphoid, idents = 16, invert = T)
DimPlot_scCustom(lymphoid)
lymphoid
lymphoid <- FindNeighbors(lymphoid, reduction = "pca", dims = 1:23)
lymphoid <- FindClusters(lymphoid, resolution = 0.7)

#Annotation of clusters after manual inspection of marker genes from the literature
new.cluster.ids <- c("B_cells","Plasma_cells","Plasma_cells","Plasma_cells", "Memory_T_cells", "ILC2s", "Naïve_CD4_T_cells", 
                     "Cd8_T_cells", "Th17_cells", "ILC3s", "Tregs", "Cycling_B_cells", "Tcr_gammadelta", "B_cells","Naïve_CD8_T_cells",
                     "Cycling_T_cells", "Plasma_cells","NKT_cells","Memory_T_cells", "B_cells")
names(new.cluster.ids) <- levels(lymphoid)
lymphoid <- RenameIdents(lymphoid, new.cluster.ids)
lymphoid$annotation <- Idents(lymphoid)
saveRDS(lymphoid, "WT12wk_DARE12Wk_Lymphoid_annotated.RDS")
#-------------------------------------------------------------------------------


#-----------------------Annotation of Myeloid cells-----------------------------
myeloid <- readRDS("../Step6_Integration/WT12wk_DARE12Wk_Integration_Myeloid_VST_without_DoubletsFromDoubletFinder.RDS")
DefaultAssay(myeloid) <- "integrated"
myeloid <- FindClusters(myeloid, resolution = 0.8)
DimPlot_scCustom(myeloid)

#Exclusion of clusters 11, 12, 13, 14, 16
myeloid <- subset(myeloid, idents = c("11", "12", "13", "14", "16"), invert = T)
myeloid <- FindNeighbors(myeloid, reduction = "pca", dims = 1:23)
myeloid <- FindClusters(myeloid, resolution = 0.5)

#Annotation of clusters after manual inspection of marker genes from the literature
new.cluster.ids <- c("Intermediate_Mfs","Granulocytes", "Granulocytes", "Activated_Mfs", "Resident_Mfs",
                     "Cdc2_a","Monocytes", "Cdc1", "Mixed_Mf_popul", "Cdc2_b", "pDCs", "Cdc2_b")
names(new.cluster.ids) <- levels(myeloid)
myeloid <- RenameIdents(myeloid, new.cluster.ids)

#exclusion of a mixed MF population
myeloid <- subset(myeloid, idents = c("Mixed_Mf_popul"), invert = T)
myeloid <- FindNeighbors(myeloid, reduction = "pca", dims = 1:23)
DimPlot_scCustom(myeloid)
myeloid$annotation <- Idents(myeloid)
saveRDS(myeloid, "WT12wk_DARE12Wk_Myeloid_annotated.RDS")
#-------------------------------------------------------------------------------

该流程从读取两个数据，一个野生型和一个实验型，分别进行质控、降维、聚类、整合；但因为在细胞注释时，代码中没有该处详细的注释，如何因为软件的差异导致注释不一致，导致代码运行出现问题。所以后面没有进行跟进follow了。详细可以见开头的GitHub链接。